In [1]:
#!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Для большей наглядности будем работать с русскоязычной версией GPT от Сбера.
# Ниже находятся команды для загрузки и инициализации модели и токенизатора.
torch.cuda.empty_cache()

In [2]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)


In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling

text = open(r'C:\Users\asus rog\Documents\parsinghoroscop\rnn\data.txt', encoding='utf-8').read()
# Сохраним обучающие данные в .txt файл 
train_path = r"C:\Users\asus rog\Documents\parsinghoroscop\rnn\data.txt"
# with open(train_path, "w") as f:
#     f.write(text)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
print(len(text))
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

33169854


C:\Users\asus rog\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [6]:
trainer.train()

***** Running training *****
  Num examples = 10083
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 7800


Step,Training Loss
500,2.989000
1000,2.628200
1500,2.433200
2000,2.281000
2500,2.154900
3000,2.046000
3500,1.957000
4000,1.881400
4500,1.816700
5000,1.762900


Saving model checkpoint to ./finetuned\checkpoint-500
Configuration saved in ./finetuned\checkpoint-500\config.json
Model weights saved in ./finetuned\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./finetuned\checkpoint-1000
Configuration saved in ./finetuned\checkpoint-1000\config.json
Model weights saved in ./finetuned\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./finetuned\checkpoint-1500
Configuration saved in ./finetuned\checkpoint-1500\config.json
Model weights saved in ./finetuned\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to ./finetuned\checkpoint-2000
Configuration saved in ./finetuned\checkpoint-2000\config.json
Model weights saved in ./finetuned\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to ./finetuned\checkpoint-2500
Configuration saved in ./finetuned\checkpoint-2500\config.json
Model weights saved in ./finetuned\checkpoint-2500\pytorch_model.bin
Saving model checkpoint to ./finetuned\checkpoint-3000
Configuration

TrainOutput(global_step=7800, training_loss=2.002399127666767, metrics={'train_runtime': 21775.6177, 'train_samples_per_second': 92.608, 'train_steps_per_second': 0.358, 'total_flos': 6.5861955477504e+16, 'train_loss': 2.002399127666767, 'epoch': 199.99})

In [7]:
print(len(text))
trainer.save_model(r'C:\Users\asus rog\Documents\parsinghoroscop\rnn\model')


Saving model checkpoint to C:\Users\asus rog\Documents\parsinghoroscop\rnn\model
Configuration saved in C:\Users\asus rog\Documents\parsinghoroscop\rnn\model\config.json


33169854


Model weights saved in C:\Users\asus rog\Documents\parsinghoroscop\rnn\model\pytorch_model.bin


In [8]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments
import torch
DEVICE = torch.device("cuda:0")
model_name_or_path = r'C:\Users\asus rog\Documents\parsinghoroscop\rnn\model'

model  = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

trainer = Trainer(model = model)


loading configuration file C:\Users\asus rog\Documents\parsinghoroscop\rnn\model\config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading weights fil

In [10]:


# Пример вероятностного сэмплирвоания с ограничением
text = "Стрельцы, "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Стрельцы,  у которых есть цели и амбиции, могут добиться успеха в чем-то быстро и легко. Главное – не бояться рисковать и соглашаться на авантюры. В отношениях сейчас царит мир, радость, любовь и взаимопонимание. Постарайтесь в этот день не спорить и не ссориться – это лишь испортит ваши отношения.
Несмотря на то, что сегодня вы настроены позитивно, день не сулит вам серьезных проблем, поэтому сконцентрируйтесь на рабочих делах. Также звезды советуют в этот день делегировать сложные задачи коллегам – так у вас получится быстрее закончить все дела. Не бойтесь делегировать сложные задачи близким – они точно справятся с ними. Также сегодня не стоит ввязываться в конфликты и спорить – вы не выиграете ни в одной игре. Полезно работать в поте лица – звезды говорят, что вы сможете добиться успеха во всем. Также полезно помогать нуждающимся – это поднимет вам настроение. А вот на личном фронте все не так радужно – возможно, некоторые отношения
